# Insect Classification Using Fine-Tuned CNN
This Colab notebook fine-tunes a pre-trained MobileNetV3Large model to classify insect species.
- Includes data augmentation
- Benchmarks CPU vs GPU performance
- Evaluates validation accuracy and loss
- Uses dataset from Kaggle

In [ ]:
# Install Kaggle API
!pip install -q kaggle

# Upload kaggle.json
from google.colab import files
files.upload()  # Upload kaggle.json here


In [ ]:
# Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download insect dataset
!kaggle datasets download -d florianblume/insect-images-classification
!unzip -q insect-images-classification.zip -d dataset


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = 'dataset/train'
val_path = 'dataset/validation'

# Data augmentation and generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.01,
    zoom_range=[0.9, 1.25],
    horizontal_flip=True,
    fill_mode='reflect'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_path, target_size=(224, 224),
                                              batch_size=32, class_mode='categorical')
val_gen = val_datagen.flow_from_directory(val_path, target_size=(224, 224),
                                          batch_size=32, class_mode='categorical')


In [ ]:
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load pre-trained model
base_model = MobileNetV3Large(input_shape=(224,224,3), include_top=False,
                              weights='imagenet', pooling='avg')
base_model.trainable = False

# Build model
model = models.Sequential([
    base_model,
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
checkpoint_cb = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy')
earlystop_cb = EarlyStopping(patience=5, restore_best_weights=True)

# Train model
history = model.fit(train_gen, validation_data=val_gen,
                    epochs=10, callbacks=[checkpoint_cb, earlystop_cb])


In [ ]:
# Evaluate model
val_loss, val_acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


In [ ]:
# Save the full model
model.save('insect_cnn_model.h5')
